In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_substr,lit,col

In [2]:
spark=SparkSession.builder.appName("line_associated_maximum_PM10").getOrCreate()

25/10/14 13:06:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
df=spark.read.text("data.txt")
df.show()

+------------------+
|             value|
+------------------+
|s1,2016-01-01,20.5|
|s2,2016-01-01,30.1|
|s1,2016-01-02,60.2|
|s2,2016-01-02,20.4|
|s1,2016-01-03,60.2|
|s2,2016-01-03,52.5|
+------------------+



In [4]:
df=df \
    .withColumn("sensor",regexp_substr("value",lit("s\d"))) \
    .withColumn("date",regexp_substr("value",lit("\d{4}-\d{2}-\d{2}"))) \
    .withColumn("score",regexp_substr("value",lit("\d{2}\.\d{1}"))) \
    .drop("value") 
df.show()

+------+----------+-----+
|sensor|      date|score|
+------+----------+-----+
|    s1|2016-01-01| 20.5|
|    s2|2016-01-01| 30.1|
|    s1|2016-01-02| 60.2|
|    s2|2016-01-02| 20.4|
|    s1|2016-01-03| 60.2|
|    s2|2016-01-03| 52.5|
+------+----------+-----+



In [5]:
max=df.agg({"score":"max"}).collect()[0]["max(score)"]
max

'60.2'

In [6]:
line=df.filter(col("score").contains(max))
line.show()

+------+----------+-----+
|sensor|      date|score|
+------+----------+-----+
|    s1|2016-01-02| 60.2|
|    s1|2016-01-03| 60.2|
+------+----------+-----+



In [9]:
line.write.mode("overwrite").csv("./output",header=True)